In [152]:
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd

In [153]:
code_df = pd.read_csv("data/stock_code/codes.csv")

In [154]:
code_df.head()

,コード,銘柄名,市場区分,本社所在地,業務分類,決算期,売買単位,支配株主等に関する事項,注意情報等,継続企業の前提の注記
0,13010,極洋,第一部,東京,水産・農林業,3月,100,無し,,無し
1,13050,ダイワ上場投信－トピックス,ETF,-,-,-,10,-,,-
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF,-,-,-,10,-,,-
3,13080,上場インデックスファンドＴＯＰＩＸ,ETF,-,-,-,100,-,,-
4,13090,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF,-,-,-,1,-,,-


In [155]:
code_df["code"] = (code_df["コード"]/10).astype(int).astype(str)
code_df["yahoo_code"] = code_df.code+".T"

In [156]:
code_df.head()

,コード,銘柄名,市場区分,本社所在地,業務分類,決算期,売買単位,支配株主等に関する事項,注意情報等,継続企業の前提の注記,code,yahoo_code
0,13010,極洋,第一部,東京,水産・農林業,3月,100,無し,,無し,1301,1301.T
1,13050,ダイワ上場投信－トピックス,ETF,-,-,-,10,-,,-,1305,1305.T
2,13060,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF,-,-,-,10,-,,-,1306,1306.T
3,13080,上場インデックスファンドＴＯＰＩＸ,ETF,-,-,-,100,-,,-,1308,1308.T
4,13090,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF,-,-,-,1,-,,-,1309,1309.T


In [157]:
import datetime
import time
sdf = None
for i,cr in code_df.iterrows():
  time.sleep(1)
  yc = cr.yahoo_code
  print("{}:{},{}".format(i,yc,cr["銘柄名"]))
  my_share = share.Share(yc)
  symbol_data = None
  try:
    symbol_data = my_share.get_historical(
#                  share.PERIOD_TYPE_DAY, 10,
#                  share.PERIOD_TYPE_DAY, 100,
                  share.PERIOD_TYPE_YEAR, 10,
#                  share.FREQUENCY_TYPE_DAY, 1)
                  share.FREQUENCY_TYPE_MONTH, 1)
  except YahooFinanceError as e:
    print("{}:{},{}".format(e.message,yc,cr["銘柄名"]))
    continue

  if symbol_data is None:
    print("{}/{} is None".format(yc,cr["銘柄名"]))
    continue

  tmpdf = pd.DataFrame(symbol_data)
  tmpdf["datetime"] = pd.to_datetime(tmpdf.timestamp, unit="ms")
  tmpdf["code_y"] = yc
  tmpdf["code"] = yc[0:4]
  ordered_tmp = tmpdf[["code","datetime","open","high","low","close","volume","code_y"]]
  if sdf is None:
    sdf = ordered_tmp
  else:
    sdf = sdf.append(ordered_tmp)
#    sdf.to_csv("data/transaction_data/trans_daily.csv",index=False)
#    sdf.to_csv("data/transaction_data/big_hist_transaction.csv",index=False)
    sdf.to_csv("data/transaction_data/trans_hist_monthly.csv",index=False)
#  if i > 10:
#     break

print(len(sdf))

0:1301.T,極洋
1:1305.T,ダイワ上場投信－トピックス
2:1306.T,ＮＥＸＴ　ＦＵＮＤＳ　ＴＯＰＩＸ連動型上場投信
3:1308.T,上場インデックスファンドＴＯＰＩＸ
4:1309.T,ＮＥＸＴ　ＦＵＮＤＳ　ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信
5:1311.T,ＮＥＸＴ　ＦＵＮＤＳ　ＴＯＰＩＸ　Ｃｏｒｅ　３０連動型上場投信
6:1312.T,ＮＥＸＴ　ＦＵＮＤＳ　ラッセル野村小型コア・インデックス連動型上場投信
7:1313.T,サムスンＫＯＤＥＸ２００証券上場指数投資信託［株式］
8:1319.T,ＮＥＸＴ　ＦＵＮＤＳ　日経３００株価指数連動型上場投信
9:1320.T,ダイワ上場投信－日経２２５
10:1321.T,ＮＥＸＴ　ＦＵＮＤＳ　日経２２５連動型上場投信
11:1322.T,上場インデックスファンド中国Ａ株（パンダ）Ｅ　Ｆｕｎｄ　ＣＳＩ３００
12:1323.T,ＮＥＸＴ　ＦＵＮＤＳ　南アフリカ株式指数・ＦＴＳＥ／ＪＳＥ　Ａｆｒｉｃａ　Ｔｏｐ４０連動型上場投信
13:1324.T,ＮＥＸＴ　ＦＵＮＤＳ　ロシア株式指数・ＲＴＳ連動型上場投信
14:1325.T,ＮＥＸＴ　ＦＵＮＤＳ　ブラジル株式指数・ボベスパ連動型上場投信
15:1326.T,ＳＰＤＲゴールド・シェア
16:1327.T,Ｓ＆Ｐ　ＧＳＣＩ商品指数　エネルギー＆メタル・キャップド・コンポーネント３５／２０・ＴＨＥＡＭ・イージーＵＣＩＴＳ・ＥＴＦクラスＡ米ドル建受益証券
17:1328.T,ＮＥＸＴ　ＦＵＮＤＳ　金価格連動型上場投信
18:1329.T,ｉシェアーズ・コア　日経２２５　ＥＴＦ
19:1330.T,上場インデックスファンド２２５
20:1332.T,日本水産
21:1333.T,マルハニチロ
22:1343.T,ＮＥＸＴ　ＦＵＮＤＳ　東証ＲＥＩＴ指数連動型上場投信
23:1344.T,ＭＡＸＩＳ　トピックス・コア３０上場投信
24:1345.T,上場インデックスファンドＪリート（東証ＲＥＩＴ指数）隔月分配型
25:1346.T,ＭＡＸＩＳ　日経２２５上場投信
26:1348.T,ＭＡＸＩＳ　トピックス上場投信
27:1349.T,ＡＢＦ汎アジア債券インデックス・ファンド
28:1352.T,ホウスイ
29:1356.T,ＴＯＰＩＸベア２倍上場

KeyboardInterrupt: 

In [ ]:
sdf.head()

,code,datetime,open,high,low,close,code_y
0,1301,2011-07-31 15:00:00,1810.0,1840.0,1660.0,1810.0,1301.T
1,1301,2011-08-31 15:00:00,1820.0,1850.0,1740.0,1840.0,1301.T
2,1301,2011-09-30 15:00:00,1810.0,1810.0,1740.0,1770.0,1301.T
3,1301,2011-10-31 15:00:00,1770.0,1930.0,1720.0,1750.0,1301.T
4,1301,2011-11-30 15:00:00,1750.0,1840.0,1740.0,1780.0,1301.T


In [ ]:
#sdf.to_csv("data/transaction_data/trans_daily.csv",index=False)
import datetime
sdf.to_csv("data/transaction_data/trans_daily_{}.csv".format(datetime.datetime.now().strftime("%Y%m%d")),index=False)